why is metakb not in the requirements?

In [1]:
# from metakb.harvesters.base import Harvester, _HarvestedData
import logging
import pandas as pd
import requests
import json
import os

In [2]:
# data = "pediatric-oncology-drug-approvals"
# url = f"https://www.fda.gov/about-fda/oncology-center-excellence/{data}"
# output_path = f"{data}.tar.gz"
# query_parameters = {"downloadformat": "tar.gz"}

In [3]:
# response = requests.get(url, stream=True, params=query_parameters)
# print(response.status_code)

In [4]:
annotated_poda_df = pd.read_csv(
    "../../../../Pediatric dataset annotations.csv", sep=","
)
annotated_poda_df = annotated_poda_df.loc[:, ~annotated_poda_df.columns.str.startswith("Unnamed")]
annotated_poda_df

,FDA Brand Label,INDICATION 2,PEDIATRIC APPROVAL DATE 3*,Variation,GENE_ID,GENETIC_LABELS,DISEASE_LABELS,DISEASE_ID
0,Asparaginase [Elspar],As a component of a multi-agent chemotherapeut...,01-10-1978,--,NaN,--,acute lymphoblastic leukemia (ALL),MONDO:0004967
1,Asparaginase Erwinia chrysanthemi (recombinant...,As a component of a multi-agent chemotherapeut...,06-30-2021,--,NaN,--,hypersensitivity to E. coli-derived asparagina...,MONDO:0005271
2,Asparaginase Erwinia chrysanthemi (recombinant...,As a component of a multi-agent chemotherapeut...,06-30-2021,--,NaN,--,lymphoblastic lymphoma (LBL),MONDO:0000873
3,Asparaginase Erwinia chrysanthemi (recombinant...,As a component of a multi-agent chemotherapeut...,06-30-2021,--,NaN,--,acute lymphoblastic leukemia (ALL),MONDO:0004967
4,Asparaginase Erwinia Chrysanthemi [Erwinaze] [...,A component of a multi-agent chemotherapeutic ...,11-18-2011,--,NaN,--,acute lymphoblastic leukemia (ALL),MONDO:0004967
...,...,...,...,...,...,...,...,...
105,Trametinib [Mekinist],"In combination with dabrafenib, for the treatm...",8-31-2023 *,V600E mutation,HGNC:1097,BRAF,metastatic neoplasm,MONDO:0024883
106,Trametinib [Mekinist],"In combination with dabrafenib, for the treatm...",03-16-2023,V600E mutation,HGNC:1097,BRAF,Low Grade Glioma,MONDO:0021637
107,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
108,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
key_map = {
    "INDICATION 2": "description",
    "GENETIC_LABELS": "geneContextQualifier",
    "FDA Brand Label": "objectTherapeutic"
}
multi_value_column = "GENETIC_LABELS"

prop_type = "VariantTherapeuticResponseProposition"
predicate = "sensitivity"

output_dir = "../../../../json_outputs"

for idx, row in annotated_poda_df.iterrows():
    values = [v.strip() for v in str(row[multi_value_column]).split(",")]

    for i, val in enumerate(values):
        json_obj = {"type": prop_type, "predicate": predicate}

        for col in key_map:
            key = key_map[col]
            value = val if col == multi_value_column else row[col]
            json_obj[key] = value

        filename = f"row_{idx}_{i}.json"
        filepath = os.path.join(output_dir, filename)

        with open(filepath, "w") as f:
            json.dump(json_obj, f, indent=4)


In [6]:
# search_string = "hypersensitivity to E.coli-derived asparaginase"

In [8]:
import pronto

In [9]:
# Load the OBO file
ontology = pronto.Ontology("http://purl.obolibrary.org/obo/mondo.obo")

URLError: <urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1000)>

In [ ]:


# Search for a term by partial name match
search_string = "Alveolar Soft Part Sarcoma"

for term in ontology.terms():
    if term.name and search_string.lower() in term.name.lower():
        print(f"ID: {term.id}")
        print(f"Name: {term.name}")
        print()

In [ ]:
# Search for a term by partial name match
search_string = "Alveolar Soft Part Sarcoma"

# First pass: look for exact match
for term in ontology.terms():
    if term.name and term.name.lower() == search_string.lower():
        exact_match = term
        break  # Stop at first exact match

# Second pass: if no exact match, look for partial matches
if exact_match:
    print("Exact match found:")
    print(f"ID: {exact_match.id}")
    print(f"Name: {exact_match.name}")
    print()
else:

In [ ]:
import pronto

# Load the OBO file
ontology = pronto.Ontology("http://purl.obolibrary.org/obo/mondo.obo")

# Search for a term by partial name match
search_string = "Alveolar Soft Part Sarcoma"

# First pass: look for exact match
for term in ontology.terms():
    if term.name and term.name.lower() == search_string.lower():
        exact_match = term
        break  # Stop at first exact match

# Second pass: if no exact match, look for partial matches
if exact_match:
    print("Exact match found:")
    print(f"ID: {exact_match.id}")
    print(f"Name: {exact_match.name}")
    print()
else:
    print("No exact match found. Searching for partial matches...\n")
    for term in ontology.terms():
        if term.name and search_string.lower() in term.name.lower():
            partial_matches.append(term)

    for term in partial_matches:
        print(f"ID: {term.id}")
        print(f"Name: {term.name}")
        print()